# Семинар 7.
# Нестандартные ситуации.

## Домашнее задание.
На основе выбранных продуктов из первого вебинара, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать Peeking Problem.   
Распишите, какие метрики и как вы будете анализировать

### Дизайн эксперимента 1: 
1. **Гипотеза**:	Запуск программы подарочных сертификатов для посетителей сайта *Sportmaster.ru* увеличит средний чек на **10%**, потому что посетители сайта будут дополнительно покупать подарочные сертификаты и использовать их в качестве подарков для своих близких и друзей.   
   
   
2. **Что делаем**:
    - Контрольная группа: Оставляем всё без изменений.
    - Тестовая группа: Выпускаем в продажу подарочные сертификаты разного номинала.    
       
       
3. **На каких пользователях тестируем**:	На всех пользователях сайта на **10%** трафика. Рандомно делим пользователей на две группы, контрольную и тестовую, для оценки влияния программы на увеличение оборота.   
 
 
4. **Ключевые метрики для оценки эксперимента**:
    - AvP (средний чек) - primary; 
    - Revenue - secondary. 
    
**Длительность эксперимента 0,004 дня** (расчёт ниже)
       
       
5. **Ожидаемый эффект**:	Увеличение среднего чека на 10%.  
 
 
6. **План действий в зависимости от результатов эксперимента**:	
    - При положительном результате эксперимента (в тестовой группе средний чек увеличивается на 10%) расширяем действие программы на всех пользователей сайта;
    - Если результат не окажет статистически значимого эффекта на увеличение среднего чека, останавливаем эксперимент.

Расситаем время проведения эксперимента:

Исторические данные: 
1. Средний трафик 7000 человек в день. 
2. Средний чек 8500 руб.
3. Стандартная ошибка 3500
4. Ожидаемый эффект - 10%
5. alpha = 5% и beta = 20%

In [132]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

In [133]:
# исходные данные
DAU = 7000
avp_1 = 8500 
std_1, std_2 = 750, 750 * 1.15
delta = 0.1
avp_2 = np.round(avp_1 + avp_1 * delta , 2)
alfa = 0.05
beta = 0.2

По имеющимся данным рассчитаем размер выборки для оценки изменений:
1. Найдем effectsize для непрерывной метрики:

In [134]:
def calc_continuous_es_alt(mean_control: Union[float, int],
                            std_control: Union[float, int],
                            mean_test: Union[float, int],
                            std_test: Union[float, int]):
    
    effect_size = (mean_test - mean_control) / ((std_control**2 + std_test**2) / 2) ** 0.5
    return effect_size

In [135]:
es_count = calc_continuous_es_alt(avp_1, std_1, avp_2, std_2)
es_count

1.05170708305214

2. Рассчитаем минимально необходимую выборку для полученного effectsize

In [136]:
def calc_sample_size_continous(effect_size: float, 
                              alpha: float = 0.05,
                              beta: float = 0.2,
                              ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size, 
                              alpha=alpha,
                              power=1-beta,
                              ratio=ratio,
                          )
    return int(n * 2)

In [137]:
print(f'Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:{calc_sample_size_continous(es_count)}')

Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:30


3. Рассчитаем длительность эксперимента в днях.    

Для этого полученное значение минимально необходимой выборки разделим на количество пользователей, посещающих сайт в день:

In [145]:
print(f'Количество дней эксперимента: {calc_sample_size_continous(es_count) / DAU}')

Количество дней эксперимента: 0.004285714285714286


### Дизайн эксперимента 2: 
1. **Гипотеза**:	Перемещение на верх сайта *Sportmaster.ru* банера с информацией о распродаже сезонных товаров повысит конверсию на 30%, потому что пользователь сможет быстрее найти все сезонные товары на сайте.   
   
   
2. **Что делаем**:
    - Контрольная группа: Оставляем всё без изменений.
    - Тестовая группа: Перемещаем блок информаци о распродаже сезонных товаров на верх первого экрана.    
       
       
3. **На каких пользователях тестируем**:	На всех пользователях сайта на **10%** трафика. Рандомно делим пользователей на две группы, контрольную и тестовую, для оценки влияния программы на увеличение оборота.   
 
 
4. **Ключевые метрики для оценки эксперимента**:
    - CR - primary; 
    - Revenue - secondary. 
    
**Длительность эксперимента 3 дня** (расчёт ниже)
       
       
5. **Ожидаемый эффект**:	увеличение количества покупок (конверсий) на 30%.  
 
 
6. **План действий в зависимости от результатов эксперимента**:	
    - При положительном результате эксперимента (в тестовой группе конверсии увеличиваются на 30%) расширяем действие программы на всех пользователей сайта;
    - Если результат не окажет статистически значимого эффекта на увеличение конверсий, останавливаем эксперимент.

Рассчитаем время проведения эксперимента:

Исторические данные: 
1. Средний трафик 7000 человек в день. 
2. Конверсия 2,5%.
4. Ожидаемый эффект 30%
5. alpha = 5% и beta = 20%

In [146]:
# исходные данные
DAU = 7000 
conv_1 = 0.025
delta = 0.3
conv_2 = conv_1 + conv_1 * delta
conv_2

0.0325

По имеющимся данным рассчитаем размер выборки для оценки изменений:
1. Найдем effectsize для дискретной метрики:

In [147]:
def calc_proportion_es(prob1: float, prob2: float):
    
    return abs(proportion_effectsize(prob1, prob2))

In [148]:
es_prop = calc_proportion_es(conv_1, conv_2)
es_prop

0.044976832996452776

2. Рассчитаем минимально необходимую выборку для полученного effectsize 

In [149]:
def calc_sample_size_proportion(effect_size: float, 
                              alpha: float = 0.05,
                              beta: float = 0.2,
                              ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size, 
                              alpha=alpha,
                              power=1-beta,
                              ratio=ratio,
                          )
    return int(n * 2)

In [150]:
print(f'Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:{calc_sample_size_proportion(es_prop)}')

Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:15519


3. Рассчитаем длительность эксперимента в днях.    

Для этого полученное значение минимально необходимой выборки разделим на количество пользователей, посещающих сайт в день:

In [151]:
print(f'Количество дней эксперимента: {calc_sample_size_proportion(es_prop) / DAU}')

Количество дней эксперимента: 2.217
